# Extract results of all runs, visualise them

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

import plotly.express as px

import DyMat

import sys
sys.path.append("../src/kissmodelica")
import postproc

## JSON input

In [5]:
resdir = "/home/oliver/temp/"

In [6]:
inputs = []
for simid in os.listdir(resdir):
    inp_name = os.path.join(resdir, simid, f"modelica_input_{simid}.json")
    with open(inp_name, encoding="utf-8") as fp:
        inputs.append(json.load(fp))
dfi = pd.DataFrame(inputs)
dfi.index = dfi.simid

In [7]:
dfi.head()

t_cover  channel_width  channel_height  \
simid                                                                          
c05bc335-87d9-11eb-9f9e-c75e935792ac   0.0020          0.008           0.001   
c05bc317-87d9-11eb-9f9e-c75e935792ac   0.0010          0.008           0.004   
c05bc2f8-87d9-11eb-9f9e-c75e935792ac   0.0005          0.016           0.001   
c05bc31b-87d9-11eb-9f9e-c75e935792ac   0.0010          0.016           0.001   
c05bc31f-87d9-11eb-9f9e-c75e935792ac   0.0010          0.016           0.002   

                                      m_flow  C_quartercell  k_contact  \
simid                                                                    
c05bc335-87d9-11eb-9f9e-c75e935792ac  0.0370           36.3      3.311   
c05bc317-87d9-11eb-9f9e-c75e935792ac  0.0370           36.3      3.311   
c05bc2f8-87d9-11eb-9f9e-c75e935792ac  0.0111           36.3      3.311   
c05bc31b-87d9-11eb-9f9e-c75e935792ac  0.0037           36.3      3.311   
c05bc31f-87d9-11eb-9f9e-c75e935792ac  0.0111           36.3      3.311   

                                      c_cover  k_cover  rho_cover  c_plate  \
simid                                                                        
c05bc335-87d9-11eb-9f9e-c75e935792ac    888.0    220.0     2700.0    888.0   
c05bc317-87d9-11eb-9f9e-c75e935792ac    888.0    220.0     2700.0    888.0   
c05bc2f8-87d9-11eb-9f9e-c75e935792ac    888.0    220.0     2700.0    888.0   
c05bc31b-87d9-11eb-9f9e-c75e935792ac    888.0    220.0     2700.0    888.0   
c05bc31f-87d9-11eb-9f9e-c75e935792ac    888.0    220.0     2700.0    888.0   

                                      ...  rho_plate  channel_dist  \
simid                                 ...                            
c05bc335-87d9-11eb-9f9e-c75e935792ac  ...     2700.0          0.04   
c05bc317-87d9-11eb-9f9e-c75e935792ac  ...     2700.0          0.04   
c05bc2f8-87d9-11eb-9f9e-c75e935792ac  ...     2700.0          0.04   
c05bc31b-87d9-11eb-9f9e-c75e935792ac  ...     2700.0          0.04   
c05bc31f-87d9-11eb-9f9e-c75e935792ac  ...     2700.0          0.04   

                                      cell_width  stoptime  area_cover  \
simid                                                                    
c05bc335-87d9-11eb-9f9e-c75e935792ac       0.043      2000    0.000344   
c05bc317-87d9-11eb-9f9e-c75e935792ac       0.043      2000    0.000344   
c05bc2f8-87d9-11eb-9f9e-c75e935792ac       0.043      2000    0.000688   
c05bc31b-87d9-11eb-9f9e-c75e935792ac       0.043      2000    0.000688   
c05bc31f-87d9-11eb-9f9e-c75e935792ac       0.043      2000    0.000688   

                                      t_plate  area_plate  area_channel  \
simid                                                                     
c05bc335-87d9-11eb-9f9e-c75e935792ac    0.032    0.000043      0.000008   
c05bc317-87d9-11eb-9f9e-c75e935792ac    0.032    0.000172      0.000032   
c05bc2f8-87d9-11eb-9f9e-c75e935792ac    0.024    0.000043      0.000016   
c05bc31b-87d9-11eb-9f9e-c75e935792ac    0.024    0.000043      0.000016   
c05bc31f-87d9-11eb-9f9e-c75e935792ac    0.024    0.000086      0.000032   

                                      perim_channel  \
simid                                                 
c05bc335-87d9-11eb-9f9e-c75e935792ac          0.018   
c05bc317-87d9-11eb-9f9e-c75e935792ac          0.024   
c05bc2f8-87d9-11eb-9f9e-c75e935792ac          0.034   
c05bc31b-87d9-11eb-9f9e-c75e935792ac          0.034   
c05bc31f-87d9-11eb-9f9e-c75e935792ac          0.036   

                                                                     simid  
simid                                                                       
c05bc335-87d9-11eb-9f9e-c75e935792ac  c05bc335-87d9-11eb-9f9e-c75e935792ac  
c05bc317-87d9-11eb-9f9e-c75e935792ac  c05bc317-87d9-11eb-9f9e-c75e935792ac  
c05bc2f8-87d9-11eb-9f9e-c75e935792ac  c05bc2f8-87d9-11eb-9f9e-c75e935792ac  
c05bc31b-87d9-11eb-9f9e-c75e935792ac  c05bc31b-87d9-11eb-9f9e-c75e935792ac  
c05bc31f-87d9-11eb-9f9e-c75

## Scalar results

Feedback: 

- füge max. fluidtemperatur hinzu

In [8]:
if 1==0:
    results = []
    for simid in os.listdir(resdir):
        res_name = os.path.join(resdir, simid, f"Kiss1_{simid}_res.mat")
        if os.path.exists(res_name):
            #print(simid)
            try:
                res = DyMat.DyMatFile(res_name)    
                # TODO: can integrate reading the input into this one
                inp = dfi.loc[simid].to_dict()
                d = postproc.extract_scalar_values2(res, nracks=9, ncells=8, **inp)
                d.update(inp)
                results.append(d)
            except ValueError:
                continue
    
    dfr = pd.DataFrame(results)
    # mass estimate: 4 * ncells * cell_width x ((2*t_cover + channel_height) x channel_dist - channel_height x channel_width)
    dfr['mass_estimate'] = 4*dfr.rho_plate*dfr.ncells*dfr.cell_width*((2*dfr.t_cover + dfr.channel_height)*dfr.channel_dist - 
                                                       dfr.channel_height * dfr.channel_width)
    dfr.to_csv("../data/108.csv", index=False)

In [9]:
dfr = pd.read_csv("../data/108.csv")
dfr.describe().T

count           mean           std            min  \
nracks          105.0       9.000000  0.000000e+00       9.000000   
ncells          105.0       8.000000  0.000000e+00       8.000000   
tend            105.0    2000.000000  0.000000e+00    2000.000000   
T_inflow        105.0     293.150000  1.142321e-13     293.150000   
Vdot            105.0       0.000018  1.438582e-05       0.000004   
v               105.0       2.429194  3.611196e+00       0.057917   
dp_max          105.0  369939.810321  1.236596e+06      62.359867   
Pmax            105.0      12.508445  4.583634e+01       0.000231   
Qgen            105.0  105849.738320  0.000000e+00  105849.738320   
T_cellmax       105.0     314.934754  8.763787e+00     303.922996   
t21             102.0     389.401961  2.426508e+02     150.000000   
tpeak           105.0      90.000000  4.805921e-13      90.000000   
T_cell1000      105.0     293.242088  2.712880e-01     293.150036   
dhyd            105.0       0.005491  3.486976e-03       0.001778   
Re              105.0    7958.307305  8.162231e+03     738.522954   
t_cover         105.0       0.001167  6.266005e-04       0.000500   
channel_width   105.0       0.009143  4.952522e-03       0.004000   
channel_height  105.0       0.001871  1.365841e-03       0.000500   
m_flow          105.0       0.017654  1.435993e-02       0.003700   
C_quartercell   105.0      36.300000  1.427901e-14      36.300000   
k_contact       105.0       3.311000  0.000000e+00       3.311000   
c_cover         105.0     888.000000  0.000000e+00     888.000000   
k_cover         105.0     220.000000  0.000000e+00     220.000000   
rho_cover       105.0    2700.000000  0.000000e+00    2700.000000   
c_plate         105.0     888.000000  0.000000e+00     888.000000   
k_plate         105.0     220.000000  0.000000e+00     220.000000   
rho_plate       105.0    2700.000000  0.000000e+00    2700.000000   
channel_dist    105.0       0.040000  6.972174e-18       0.040000   
cell_width      105.0       0.043000  0.000000e+00       0.043000   
stoptime        105.0    2000.000000  0.000000e+00    2000.000000   
area_cover      105.0       0.000393  2.129584e-04       0.000172   
t_plate         105.0       0.030857  4.952522e-03       0.024000   
area_plate      105.0       0.000080  5.873117e-05       0.000022   
area_channel    105.0       0.000017  1.713527e-05       0.000002   
perim_channel   105.0       0.022029  1.026519e-02       0.009000   
mass_estimate   105.0       0.561384  2.465031e-01       0.193190   

                          25%            50%            75%           max  
nracks               9.000000       9.000000       9.000000  9.000000e+00  
ncells               8.000000       8.000000       8.000000  8.000000e+00  
tend              2000.000000    2000.000000    2000.000000  2.000000e+03  
T_inflow           293.150000     293.150000     293.150000  2.931500e+02  
Vdot                 0.000004       0.000011       0.000037  3.706672e-05  
v                    0.463334       1.158335       2.780004  1.853336e+01  
dp_max            2873.523283   20021.413337  128425.408246  7.168939e+06  
Pmax                 0.013481       0.222638       2.347148  2.657291e+02  
Qgen            105849.738320  105849.738320  105849.738320  1.058497e+05  
T_cellmax          304.402708     315.915544     323.266352  3.284567e+02  
t21                159.000000     273.000000     703.750000  7.700000e+02  
tpeak               90.000000      90.000000      90.000000  9.000000e+01  
T_cell1000         293.152281     293.164101     293.209807  2.947319e+02  
dhyd                 0.001939       0.003765       0.008000  1.280000e-02  
Re                2215.568862    4923.486361    9846.972721  3.282324e+04  
t_cover              0.000500       0.001000       0.002000  2.000000e-03  
channel_width        0.004000       0.008000       0.016000  1.600000e-02  
channel_height       0.000500       0.001000       0.004000  4.000000e-03  
m_flow            

In [10]:
# just keep relevant columns (having std dev above some limit)
cols = ['simid', 'Vdot', 'v', 'dp_max', 'Pmax', 'T_cellmax', 't21', 'T_cell1000', 'dhyd',
        'Re', 't_cover', 'channel_width', 'channel_height', 'm_flow', 'area_cover',
        't_plate', 'area_plate', 'area_channel', 'perim_channel', 'mass_estimate']
dfrr = dfr[cols]

# convert to degC
dfrr.loc[:,'T_cellmax'] = dfrr.loc[:,'T_cellmax'].values - 273.15
dfrr.loc[:,'T_cell1000'] = dfrr.loc[:,'T_cell1000'].values - 273.15

# compute channel aspect ratio
dfrr["aspect_ratio"] = dfrr["channel_width"]/dfrr["channel_height"]

/usr/lib/python3/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
<ipython-input-10-60c2c51215ea>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfrr["aspect_ratio"] = dfrr["channel_width"]/dfrr["channel_height"]


In [11]:
dfrr

simid      Vdot         v        dp_max  \
0    c05bc335-87d9-11eb-9f9e-c75e935792ac  0.000037  4.633340  2.699454e+05   
1    c05bc317-87d9-11eb-9f9e-c75e935792ac  0.000037  1.158335  6.089612e+03   
2    c05bc2f8-87d9-11eb-9f9e-c75e935792ac  0.000011  0.695001  8.650348e+03   
3    c05bc31b-87d9-11eb-9f9e-c75e935792ac  0.000004  0.231667  2.883502e+03   
4    c05bc31f-87d9-11eb-9f9e-c75e935792ac  0.000011  0.347501  1.212276e+03   
..                                    ...       ...       ...           ...   
100  c05bc30b-87d9-11eb-9f9e-c75e935792ac  0.000037  2.316670  2.906510e+04   
101  c05bc345-87d9-11eb-9f9e-c75e935792ac  0.000004  0.057917  6.236003e+01   
102  c05bc2f2-87d9-11eb-9f9e-c75e935792ac  0.000011  0.347501  5.455934e+02   
103  c05bc2ea-87d9-11eb-9f9e-c75e935792ac  0.000037  9.266680  2.004880e+06   
104  c05bc2fb-87d9-11eb-9f9e-c75e935792ac  0.000011  0.347501  1.212275e+03   

          Pmax  T_cellmax    t21  T_cell1000      dhyd            Re  ...  \
0    10.005989  31.142984  155.0   20.059807  0.003556  16411.621202  ...   
1     0.225722  30.906573  153.0   20.001353  0.010667  12308.715902  ...   
2     0.096192  42.833326  260.0   20.002076  0.003765   2606.551603  ...   
3     0.010688  52.802723  709.0   20.012309  0.003765    868.850534  ...   
4     0.013481  42.404132  273.0   20.000277  0.007111   2461.743180  ...   
..         ...        ...    ...         ...       ...           ...  ...   
100   1.077348  31.088933  155.0   20.006455  0.008000  18463.073852  ...   
101   0.000231  47.409167  770.0   20.152377  0.012800    738.522954  ...   
102   0.006067  42.055511  285.0   20.000167  0.010667   3692.614770  ...   
103  74.314331  31.252708  171.0   20.443653  0.001882  17377.010685  ...   
104   0.013481  42.480962  270.0   20.000287  0.007111   2461.743180  ...   

     channel_width  channel_height  m_flow  area_cover  t_plate  area_plate  \
0            0.008          0.0010  0.0370    0.000344    0.032    0.000043   
1            0.008          0.0040  0.0370    0.000344    0.032    0.000172   
2            0.016          0.0010  0.0111    0.000688    0.024    0.000043   
3            0.016          0.0010  0.0037    0.000688    0.024    0.000043   
4            0.016          0.0020  0.0111    0.000688    0.024    0.000086   
..             ...             ...     ...         ...      ...         ...   
100          0.004          0.0040  0.0370    0.000172    0.036    0.000172   
101          0.016          0.0040  0.0037    0.000688    0.024    0.000172   
102          0.008          0.0040  0.0111    0.000344    0.032    0.000172   
103          0.008          0.0005  0.0370    0.000344    0.032    0.000022   
104          0.016          0.0020  0.0111    0.000688    0.024    0.000086   

     area_channel  perim_channel  mass_estimate  aspect_ratio  
0        0.000008          0.018       0.713318           8.0  
1        0.000032          0.024       0.772762           2.0  
2        0.000016          0.034       0.237773          16.0  
3        0.000016          0.034       0.386381          16.0  
4        0.000032          0.036       0.475546           8.0  
..            ...            ...            ...           ...  
100      0.000016          0.016       0.832205           1.0  
101      0.000064          0.040       0.951091           4.0  
102      0.000032          0.024       0.624154           2.0  
103      0.000004          0.017       0.208051          16.0  
104      0.000032          0.036       0.326938           8.0  

[105 rows x 21 columns]

In [12]:
fig = px.scatter(dfrr, x="mass_estimate", y="t21", color='m_flow', 
                 hover_data={c:True for c in cols})
fig.show()

In [13]:
#dfrr.plot(x="mass_estimate", y="T_cellmax", kind="scatter");
fig = px.scatter(dfrr, x="mass_estimate", y="T_cellmax", color='m_flow', 
                 hover_data={c:True for c in cols})
fig.show()

In [14]:
# da scheint es eine optimale strömungsgeschwindigkeit zu geben
fig = px.scatter(dfrr, x="v", y="T_cell1000", color='m_flow', log_x=True, log_y=True,
                 hover_data={c:True for c in cols}) 
fig.show()

In [15]:
#dfrr.plot(x="mass_estimate", y="Pmax", kind="scatter");
fig = px.scatter(dfrr, x="mass_estimate", y="dp_max", color='m_flow', log_y=True,
                 hover_data={c:True for c in cols})
fig.show()

In [16]:
fig = px.scatter(dfrr, x="Re", y="dp_max", color='m_flow', log_y=True, log_x=True,
                 hover_data={c:True for c in cols})
fig.show()

In [17]:
fig = px.scatter(dfrr, x="aspect_ratio", y="mass_estimate", color='m_flow', log_y=False, log_x=False,
                 hover_data={c:True for c in cols})
fig.show()